In [1]:
from pymongo import MongoClient
import pandas as pd
import threading
import os

#import winsound
Freq = 2500 # Set Frequency To 2500 Hertz
Dur = 1000 # Set Duration To 1000 ms == 1 second
db=MongoClient('10.247.69.18').crowd
global n0
n0=db['gofundme_donated'].count()

def make_df(sort,n):
    cursor = db['gofundme_donated'].find({},{"url":1,"_id":0},limit =n).sort("_id", sort)
    df =  pd.DataFrame(list(cursor))
    df.columns=['url']
    return df
    
def if_new_donation():
  global n0
  global all
  threading.Timer(5.0, if_new_donation).start()
  n=db['gofundme_donated'].count()
  d=n-n0  
  if d> 0:
    df=make_df(-1,d)
    for i in range(d):
        url=df.url[i]
        print n,n0,d,i,url
        if url in all.url.tolist():
            #winsound.Beep(Freq,Dur)
            c_donate=all.loc[all['url'] == url].c_donate.values[0]
            c_donate=c_donate+1
            all = all[all.url != url]
            all=all.append(pd.DataFrame([[url,c_donate]],columns=['url','c_donate']))
            db['gofundme_url'].delete_one({"url":url})
            db['gofundme_url'].insert_one({"url":url,"c_donate":c_donate})
        else: 
            print 'else'
            db.gofundme_url.insert_one({"url":url,"c_donate":1})
            all=all.append(pd.DataFrame([[url,1]],columns=['url','c_donate']))            
    n0=n
    #os.remove('urls.csv')
    all.to_csv('urls.csv',index=False)
    
cursor = db['gofundme_donated'].find({},{"url":1,"_id":0},limit =10000)
if len(list(cursor))==0:
    all=pd.DataFrame(columns=['url','c_donate'])
else:
    cursor=db['gofundme_url'].find({},{"url":1,"c_donate":1,"_id":0},limit =10000)
    all=pd.DataFrame(list(cursor),columns=['url','c_donate'])
    
if_new_donation()

In [8]:
all.sort('c_donate',ascending=False)

489 488 1 0 https://gofund.me/aronskidney
else


,url,c_donate
1,https://gofund.me/s36wweyg,3
0,https://gofund.me/z5g4z9d8,2
0,https://gofund.me/amg89mgs,1
3,https://gofund.me/HelpKeepJimsFoot,1
4,https://gofund.me/teamkevo,1
5,https://gofund.me/blanketssd,1
6,https://gofund.me/vwfvshtw,1
7,https://gofund.me/g6e7gwmc,1
8,https://gofund.me/9gfrtcss,1
0,https://gofund.me/aronskidney,1
